In [1]:
import pandas as pd
import numpy as np
import itertools

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
from imblearn.metrics import *

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn.neural_network import MLPClassifier


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt

In [2]:
data= pd.read_csv('mendeley_abd.csv')
data

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,45,17,0,2,0,0,0,0,0,...,0,0,0,448,5396,3980,0,0,6,legitimate
11426,http://www.budgetbots.com/server.php/Server%20...,84,18,0,5,0,1,1,0,0,...,1,0,0,211,6728,0,0,1,0,phishing
11427,https://www.facebook.com/Interactive-Televisio...,105,16,1,2,6,0,1,0,0,...,0,0,0,2809,8515,8,0,1,10,legitimate
11428,http://www.mypublicdomainpictures.com/,38,30,0,2,0,0,0,0,0,...,1,0,0,85,2836,2455493,0,0,4,legitimate


# Data preparation

In [3]:
data=data.drop('url',axis=1)
data

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,37,19,0,3,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,77,23,1,1,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,126,50,1,4,1,0,1,2,0,3,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,18,11,0,2,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,55,15,0,2,2,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,45,17,0,2,0,0,0,0,0,0,...,0,0,0,448,5396,3980,0,0,6,legitimate
11426,84,18,0,5,0,1,1,0,0,1,...,1,0,0,211,6728,0,0,1,0,phishing
11427,105,16,1,2,6,0,1,0,0,1,...,0,0,0,2809,8515,8,0,1,10,legitimate
11428,38,30,0,2,0,0,0,0,0,0,...,1,0,0,85,2836,2455493,0,0,4,legitimate


In [4]:
data['status'].replace('legitimate',0,inplace=True)
data

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,37,19,0,3,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,0
1,77,23,1,1,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,126,50,1,4,1,0,1,2,0,3,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,18,11,0,2,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,0
4,55,15,0,2,2,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,45,17,0,2,0,0,0,0,0,0,...,0,0,0,448,5396,3980,0,0,6,0
11426,84,18,0,5,0,1,1,0,0,1,...,1,0,0,211,6728,0,0,1,0,phishing
11427,105,16,1,2,6,0,1,0,0,1,...,0,0,0,2809,8515,8,0,1,10,0
11428,38,30,0,2,0,0,0,0,0,0,...,1,0,0,85,2836,2455493,0,0,4,0


In [5]:
data['status'].replace('phishing',1,inplace=True)
data

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,37,19,0,3,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,0
1,77,23,1,1,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,1
2,126,50,1,4,1,0,1,2,0,3,...,1,0,0,14,4004,5828815,0,1,0,1
3,18,11,0,2,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,0
4,55,15,0,2,2,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,45,17,0,2,0,0,0,0,0,0,...,0,0,0,448,5396,3980,0,0,6,0
11426,84,18,0,5,0,1,1,0,0,1,...,1,0,0,211,6728,0,0,1,0,1
11427,105,16,1,2,6,0,1,0,0,1,...,0,0,0,2809,8515,8,0,1,10,0
11428,38,30,0,2,0,0,0,0,0,0,...,1,0,0,85,2836,2455493,0,0,4,0


In [6]:
np.any(data.isnull().sum())

False

# train-test split

In [7]:
x= data.drop('status',axis=1)
x

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,37,19,0,3,0,0,0,0,0,0,...,0,0,1,0,45,-1,0,1,1,4
1,77,23,1,1,0,0,0,0,0,0,...,0,1,0,0,77,5767,0,0,1,2
2,126,50,1,4,1,0,1,2,0,3,...,0,1,0,0,14,4004,5828815,0,1,0
3,18,11,0,2,0,0,0,0,0,0,...,0,1,0,0,62,-1,107721,0,0,3
4,55,15,0,2,2,0,0,0,0,0,...,0,0,1,0,224,8175,8725,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,45,17,0,2,0,0,0,0,0,0,...,0,0,0,0,448,5396,3980,0,0,6
11426,84,18,0,5,0,1,1,0,0,1,...,0,1,0,0,211,6728,0,0,1,0
11427,105,16,1,2,6,0,1,0,0,1,...,0,0,0,0,2809,8515,8,0,1,10
11428,38,30,0,2,0,0,0,0,0,0,...,0,1,0,0,85,2836,2455493,0,0,4


In [8]:
y=data['status']
y

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: int64

In [9]:
x_train=x
y_train=y

In [10]:
y.value_counts()

0    5715
1    5715
Name: status, dtype: int64

In [11]:
sc= StandardScaler()
x_train=sc.fit_transform(x)
x_train

array([[-0.43632748, -0.19396372, -0.42102044, ...,  6.97822701,
         0.93426411,  0.32097385],
       [ 0.28706655,  0.17720743,  2.37518157, ..., -0.14330288,
         0.93426411, -0.46740717],
       [ 1.17322424,  2.6826127 ,  2.37518157, ..., -0.14330288,
         0.93426411, -1.25578819],
       ...,
       [ 0.79344237, -0.47234208,  2.37518157, ..., -0.14330288,
         0.93426411,  2.68611691],
       [-0.41824263,  0.82675695, -0.42102044, ..., -0.14330288,
        -1.07036115,  0.32097385],
       [ 7.52100688, -0.65792766,  2.37518157, ...,  6.97822701,
         0.93426411, -1.25578819]])

# defining models

In [12]:
knn= KNeighborsClassifier(n_neighbors=10,n_jobs=-1)
svm= SVC(random_state=10,kernel='rbf')
lr = LogisticRegression()

rf= RandomForestClassifier(n_jobs=-1,random_state=10)
ada= AdaBoostClassifier(random_state=100)
extra = ExtraTreesClassifier(n_jobs=-1,random_state=10)

lgbm = LGBMClassifier(n_jobs=-1,random_state=10)
grad = GradientBoostingClassifier()
xg = XGBClassifier(use_label_encoder =False, eval_metric='logloss')

nn= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (64,32,16,8), random_state=10, max_iter=500)

In [13]:
classifiers= {
    "RandomForest": rf,
    "ExtraTree": extra,
    "gradBoost": grad,
    "adaboost": ada,
    "LGBM": lgbm,
    "xgb": xg,
    "nn":nn}

In [14]:
classifier_2 = nn

In [15]:
scores={'accuracy': make_scorer(accuracy_score),
        'recall' : make_scorer(recall_score),
        'precision':make_scorer(precision_score),
        'gmean': make_scorer(geometric_mean_score),
        'f1':make_scorer(f1_score),
        'roc': make_scorer(roc_auc_score)
        }

# training

In [19]:
models_scores_results_10, models_names_10 = list(), list() 

In [20]:
rf_result = cross_validate(rf, x_train, y_train, cv = 10, scoring=scores,n_jobs=-1)
df= pd.DataFrame(rf_result)
df.mean(axis=0)

fit_time          0.905705
score_time        0.196944
test_accuracy     0.966492
test_recall       0.964655
test_precision    0.968240
test_gmean        0.966486
test_f1           0.966436
test_roc          0.966492
dtype: float64

In [21]:
ada_result= cross_validate(ada, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(ada_result)
df.mean(axis=0)

fit_time          2.144392
score_time        0.018105
test_accuracy     0.946369
test_recall       0.943135
test_precision    0.949348
test_gmean        0.946342
test_f1           0.946188
test_roc          0.946372
dtype: float64

In [22]:
extra_result= cross_validate(extra, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(extra_result)
df.mean(axis=0)

fit_time          0.821086
score_time        0.223950
test_accuracy     0.965879
test_recall       0.960279
test_precision    0.971166
test_gmean        0.965860
test_f1           0.965687
test_roc          0.965879
dtype: float64

In [23]:
grad_result= cross_validate(grad, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(grad_result)
df.mean(axis=0)

fit_time          6.614569
score_time        0.005702
test_accuracy     0.957130
test_recall       0.955731
test_precision    0.958477
test_gmean        0.957115
test_f1           0.957075
test_roc          0.957130
dtype: float64

In [24]:
lgbm_result= cross_validate(lgbm, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(lgbm_result)
df.mean(axis=0)

fit_time          0.590735
score_time        0.011202
test_accuracy     0.968416
test_recall       0.968153
test_precision    0.968703
test_gmean        0.968405
test_f1           0.968407
test_roc          0.968416
dtype: float64

In [25]:
xg_result= cross_validate(xg, x_train, y, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(xg_result)
df.mean(axis=0)

fit_time          4.850106
score_time        0.010602
test_accuracy     0.970779
test_recall       0.972178
test_precision    0.969481
test_gmean        0.970773
test_f1           0.970820
test_roc          0.970779
dtype: float64

In [28]:
nn_result= cross_validate(nn, x_train, y, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(nn_result)
df.mean(axis=0)

fit_time          5.984038
score_time        0.006802
test_accuracy     0.955993
test_recall       0.955904
test_precision    0.956150
test_gmean        0.955972
test_f1           0.955996
test_roc          0.955990
dtype: float64

In [16]:
nb= GaussianNB()
nb_result= cross_validate(nb, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(nb_result)
df.mean(axis=0)

fit_time          0.040808
score_time        0.008702
test_accuracy     0.686964
test_recall       0.398762
test_precision    0.941137
test_gmean        0.623271
test_f1           0.559772
test_roc          0.686957
dtype: float64

In [17]:
knn_result= cross_validate(knn, x_train, y, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(knn_result)
df.mean(axis=0)

fit_time          0.079468
score_time        1.478134
test_accuracy     0.934471
test_recall       0.908845
test_precision    0.957973
test_gmean        0.934095
test_f1           0.932710
test_roc          0.934475
dtype: float64

In [18]:
svm_result= cross_validate(svm, x_train, y, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(svm_result)
df.mean(axis=0)

fit_time          19.169036
score_time         0.423267
test_accuracy      0.959055
test_recall        0.955553
test_precision     0.962310
test_gmean         0.959039
test_f1            0.958903
test_roc           0.959055
dtype: float64

In [19]:
lr_result= cross_validate(lr, x_train, y, cv = 10, scoring=scores,  n_jobs=-1)
df= pd.DataFrame(lr_result)
df.mean(axis=0)

fit_time          1.564678
score_time        0.009193
test_accuracy     0.944794
test_recall       0.940684
test_precision    0.948538
test_gmean        0.944761
test_f1           0.944550
test_roc          0.944795
dtype: float64

# Stacking - probability based

In [29]:
# meta - nn,  cv - 10


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6,12,24,12,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg, nn]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [30]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          64.504889
score_time         0.097322
test_accuracy      0.970691
test_recall        0.968152
test_precision     0.973141
test_gmean         0.970677
test_f1            0.970623
test_roc           0.970689
dtype: float64

In [31]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6,12,24,12,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [32]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          45.890189
score_time         0.082087
test_accuracy      0.969641
test_recall        0.966054
test_precision     0.973090
test_gmean         0.969623
test_f1            0.969536
test_roc           0.969642
dtype: float64

# stacking - optimize meta classifier

In [33]:
sel_classifier= [rf, ada, extra, lgbm, grad, xg]

nn_basic = MLPClassifier(random_state=10,verbose = True)

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_basic, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

params= {
    'meta_classifier__hidden_layer_sizes': [(12,12,12), (6,6,6),(6,12,6),(12,6,6),(6,12,32,12,6)],
}

grid = GridSearchCV(estimator=scl, 
                    param_grid=params, 
                    cv=10,
                    refit=True,
                    n_jobs=-1)

grid.fit(x_train, y_train)


Iteration 1, loss = 0.60876324
Iteration 2, loss = 0.32914712
Iteration 3, loss = 0.14624617
Iteration 4, loss = 0.11096668
Iteration 5, loss = 0.10661026
Iteration 6, loss = 0.10463267
Iteration 7, loss = 0.10339997
Iteration 8, loss = 0.10219464
Iteration 9, loss = 0.10156494
Iteration 10, loss = 0.10059000
Iteration 11, loss = 0.09969572
Iteration 12, loss = 0.09898634
Iteration 13, loss = 0.09882547
Iteration 14, loss = 0.09818112
Iteration 15, loss = 0.09836699
Iteration 16, loss = 0.09758810
Iteration 17, loss = 0.09746250
Iteration 18, loss = 0.09745528
Iteration 19, loss = 0.09691688
Iteration 20, loss = 0.09663269
Iteration 21, loss = 0.09634811
Iteration 22, loss = 0.09667943
Iteration 23, loss = 0.09624060
Iteration 24, loss = 0.09592439
Iteration 25, loss = 0.09689158
Iteration 26, loss = 0.09693015
Iteration 27, loss = 0.09554550
Iteration 28, loss = 0.09556888
Iteration 29, loss = 0.09550040
Iteration 30, loss = 0.09648798
Iteration 31, loss = 0.09514745
Iteration 32, los

GridSearchCV(cv=10,
             estimator=StackingCVClassifier(classifiers=[RandomForestClassifier(n_jobs=-1,
                                                                                random_state=10),
                                                         AdaBoostClassifier(random_state=100),
                                                         ExtraTreesClassifier(n_jobs=-1,
                                                                              random_state=10),
                                                         LGBMClassifier(random_state=10),
                                                         GradientBoostingClassifier(),
                                                         XGBClassifier(base_score=None,
                                                                       booster=None,
                                                                       colsample_bylevel=None,
                                                                  

In [34]:
df= pd.DataFrame(grid.cv_results_)
df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_meta_classifier__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4,74.839363,16.776806,0.086697,0.019968,"(6, 12, 32, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.971129,0.972878,0.968504,0.965879,0.974628,0.972878,0.965879,0.965879,0.979003,0.973753,0.971041,0.004204,1
2,87.301148,4.697802,0.130230,0.029755,"(6, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.972003,0.972003,0.968504,0.967629,0.971129,0.974628,0.965004,0.960630,0.978128,0.972878,0.970254,0.004744,2
1,85.286885,10.138808,0.133230,0.048146,"(6, 6, 6)","{'meta_classifier__hidden_layer_sizes': (6, 6,...",0.971129,0.971129,0.967629,0.967629,0.972878,0.968504,0.966754,0.963255,0.979003,0.972878,0.970079,0.004118,3
3,88.520422,6.415499,0.164837,0.080393,"(12, 6, 6)","{'meta_classifier__hidden_layer_sizes': (12, 6...",0.972003,0.972003,0.966754,0.965004,0.970254,0.972878,0.965879,0.963255,0.979003,0.972878,0.969991,0.004513,4
0,82.969856,7.545000,0.152135,0.059236,"(12, 12, 12)","{'meta_classifier__hidden_layer_sizes': (12, 1...",0.972003,0.971129,0.966754,0.964129,0.972003,0.972878,0.965004,0.965004,0.978128,0.972003,0.969904,0.004272,5


In [35]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6,12,32,12,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg, nn]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [36]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          72.605932
score_time         0.105275
test_accuracy      0.971041
test_recall        0.964828
test_precision     0.977013
test_gmean         0.971007
test_f1            0.970858
test_roc           0.971040
dtype: float64

# feature selection

In [37]:
from sklearn.feature_selection import RFECV

In [41]:
selector = RFECV(rf, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
rf_col,= np.where(selector.ranking_==1)
rf_col

array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 17, 19, 20,
       21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58,
       60, 62, 64, 65, 66, 67, 69, 70, 73, 74, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86], dtype=int64)

In [42]:
selector = RFECV(ada, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
ada_col,= np.where(selector.ranking_==1)
ada_col

array([ 0,  1,  2,  3,  4,  6, 10, 13, 19, 20, 24, 26, 35, 39, 40, 41, 43,
       44, 46, 47, 49, 50, 51, 54, 56, 57, 58, 62, 64, 70, 78, 81, 82, 83,
       85, 86], dtype=int64)

In [43]:
selector = RFECV(extra, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
extra_col= np.where(selector.ranking_==1)
extra_col

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 17, 19, 20,
        21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40,
        41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58,
        60, 62, 64, 65, 66, 67, 69, 70, 72, 73, 74, 77, 78, 79, 80, 81, 82,
        83, 84, 85, 86], dtype=int64),)

In [44]:
selector = RFECV(lgbm, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
lgbm_col= np.where(selector.ranking_==1)
lgbm_col

(array([ 0,  1,  2,  3,  4,  5,  6,  9, 10, 11, 12, 13, 15, 17, 19, 20, 21,
        22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 37, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 60, 62, 64,
        65, 66, 67, 69, 70, 73, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86],
       dtype=int64),)

In [45]:
selector = RFECV(grad, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
grad_col= np.where(selector.ranking_==1)
grad_col

(array([ 0,  1,  2,  3,  4,  6,  9, 10, 12, 13, 19, 20, 23, 24, 25, 26, 28,
        35, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 56, 57,
        58, 60, 62, 66, 70, 74, 77, 78, 79, 81, 82, 83, 85, 86],
       dtype=int64),)

In [46]:
selector = RFECV(xg, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y)
xg_col= np.where(selector.ranking_==1)
xg_col

(array([ 0,  1,  2,  3,  4,  5,  6,  9, 10, 13, 19, 20, 24, 25, 26, 35, 37,
        39, 40, 42, 43, 44, 46, 47, 49, 50, 51, 54, 56, 57, 58, 62, 64, 65,
        66, 67, 74, 77, 78, 79, 81, 82, 83, 85, 86], dtype=int64),)

# stacking with subsets of features

In [47]:
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline

In [48]:
pipe_rf = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 17, 19, 20,
       21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58,
       60, 62, 64, 65, 66, 67, 69, 70, 73, 74, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86)),rf)

In [49]:
pipe_ada = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  6, 10, 13, 19, 20, 24, 26, 35, 39, 40, 41, 43,
       44, 46, 47, 49, 50, 51, 54, 56, 57, 58, 62, 64, 70, 78, 81, 82, 83,
       85, 86)),ada)

In [50]:
pipe_extra = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 17, 19, 20,
        21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40,
        41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58,
        60, 62, 64, 65, 66, 67, 69, 70, 72, 73, 74, 77, 78, 79, 80, 81, 82,
        83, 84, 85, 86)),extra)

In [51]:
pipe_lgbm = make_pipeline(ColumnSelector(cols=( 0,  1,  2,  3,  4,  5,  6,  9, 10, 11, 12, 13, 15, 17, 19, 20, 21,
        22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 37, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 60, 62, 64,
        65, 66, 67, 69, 70, 73, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86)),lgbm)

In [52]:
pipe_grad = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  6,  9, 10, 12, 13, 19, 20, 23, 24, 25, 26, 28,
        35, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 56, 57,
        58, 60, 62, 66, 70, 74, 77, 78, 79, 81, 82, 83, 85, 86)),grad)

In [53]:
pipe_xg = make_pipeline(ColumnSelector(cols=( 0,  1,  2,  3,  4,  5,  6,  9, 10, 13, 19, 20, 24, 25, 26, 35, 37,
        39, 40, 42, 43, 44, 46, 47, 49, 50, 51, 54)),xg)

In [54]:
pipe_nn = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  5,  6,  9, 10, 11, 12, 13, 15, 17, 19, 20, 21,
        22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 37, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 60, 62, 64,
        65, 66, 67, 69, 70, 73, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86)),nn)

In [55]:
nn= MLPClassifier(activation = "relu", hidden_layer_sizes = (6, 12, 32, 12, 6), random_state=10)

sclf = StackingCVClassifier(classifiers=[pipe_rf, pipe_ada, pipe_extra, pipe_lgbm, pipe_grad, pipe_xg, pipe_nn], 
                            meta_classifier=nn,
                            use_probas = True,
                            random_state=42)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)

In [56]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          64.680356
score_time         0.094121
test_accuracy      0.971129
test_recall        0.965702
test_precision     0.976318
test_gmean         0.971104
test_f1            0.970968
test_roc           0.971127
dtype: float64

In [48]:
sel_classifier= [pipe_rf, pipe_ada, pipe_extra, pipe_lgbm, pipe_grad, pipe_xg]

nn = MLPClassifier(random_state=10,verbose = True)

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

params= {
    'meta_classifier__hidden_layer_sizes': [(24,32,24), (6,6,6),(12,6,6),(6,12,32,12,6),(6,12,24,36,24,12,6)],
}

grid = GridSearchCV(estimator=scl, 
                    param_grid=params, 
                    cv=10,
                    refit=True,
                    n_jobs=-1)

grid.fit(x_train, y_train)


Iteration 1, loss = 0.71568247
Iteration 2, loss = 0.49040722
Iteration 3, loss = 0.25149136
Iteration 4, loss = 0.12221053
Iteration 5, loss = 0.07922964
Iteration 6, loss = 0.06550729
Iteration 7, loss = 0.06022840
Iteration 8, loss = 0.05808860
Iteration 9, loss = 0.05682885
Iteration 10, loss = 0.05610499
Iteration 11, loss = 0.05562725
Iteration 12, loss = 0.05515154
Iteration 13, loss = 0.05486390
Iteration 14, loss = 0.05460022
Iteration 15, loss = 0.05444276
Iteration 16, loss = 0.05417749
Iteration 17, loss = 0.05395469
Iteration 18, loss = 0.05369328
Iteration 19, loss = 0.05358234
Iteration 20, loss = 0.05336230
Iteration 21, loss = 0.05325823
Iteration 22, loss = 0.05309899
Iteration 23, loss = 0.05300421
Iteration 24, loss = 0.05288205
Iteration 25, loss = 0.05294038
Iteration 26, loss = 0.05278838
Iteration 27, loss = 0.05267996
Iteration 28, loss = 0.05261776
Iteration 29, loss = 0.05230492
Iteration 30, loss = 0.05237170
Iteration 31, loss = 0.05198887
Iteration 32, los

GridSearchCV(cv=10,
             estimator=StackingCVClassifier(classifiers=[Pipeline(steps=[('columnselector',
                                                                          ColumnSelector(cols=(0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5,
                                                                                               7,
                                                                                               8,
                                                                                               9,
      

In [49]:
df= pd.DataFrame(grid.cv_results_)
df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_meta_classifier__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,38.844759,2.789992,0.182825,0.166115,"(12, 6, 6)","{'meta_classifier__hidden_layer_sizes': (12, 6...",0.985,0.978,0.990,0.980,0.983,0.992,0.979,0.973,0.980,0.983,0.9823,0.005367,1
3,40.501064,1.640083,0.093757,0.009883,"(6, 12, 32, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.984,0.978,0.991,0.980,0.979,0.992,0.980,0.977,0.977,0.984,0.9822,0.005212,2
1,39.941713,3.209237,0.104037,0.018282,"(6, 6, 6)","{'meta_classifier__hidden_layer_sizes': (6, 6,...",0.984,0.979,0.987,0.978,0.985,0.992,0.982,0.969,0.979,0.984,0.9819,0.005873,3
4,35.771682,5.721634,0.071899,0.017410,"(6, 12, 24, 36, 24, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.988,0.980,0.989,0.978,0.983,0.990,0.976,0.973,0.979,0.983,0.9819,0.005449,3
0,28.187922,10.562209,0.123499,0.064328,"(24, 32, 24)","{'meta_classifier__hidden_layer_sizes': (24, 3...",0.983,0.979,0.991,0.980,0.982,0.992,0.969,0.974,0.980,0.984,0.9814,0.006576,5


In [50]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (12,6,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [51]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          23.076894
score_time         0.082759
test_accuracy      0.981700
test_recall        0.977800
test_precision     0.985564
test_gmean         0.981676
test_f1            0.981636
test_roc           0.981700
dtype: float64

In [52]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6, 12, 32, 12, 6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [53]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          22.250925
score_time         0.089071
test_accuracy      0.981500
test_recall        0.975400
test_precision     0.987505
test_gmean         0.981461
test_f1            0.981375
test_roc           0.981500
dtype: float64